In [ ]:
import yfinance as yf
import backtrader as bt
from datetime import datetime
import pandas as pd

In [ ]:

# ETF-Daten abrufen und vorbereiten
def fetch_etf_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"Keine Daten für {ticker} im angegebenen Zeitraum gefunden.")
    # Backtrader benötigt spezifische Spaltennamen
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    data.columns = ['open', 'high', 'low', 'close', 'volume']
    data['openinterest'] = 0  # Backtrader benötigt diese Spalte
    return data

# SMA-Strategie definieren
class SmaStrategy(bt.Strategy):
    params = (
        ("short_sma", 10),  # Kurzfristige SMA-Periode
        ("long_sma", 50),   # Langfristige SMA-Periode
    )

    def __init__(self):
        # SMA-Indikatoren berechnen
        self.short_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_sma)
        self.long_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_sma)

    def next(self):
        # Kaufen, wenn kurz über lang
        if self.short_sma > self.long_sma and not self.position:
            self.buy()
        # Verkaufen, wenn lang über kurz
        elif self.short_sma < self.long_sma and self.position:
            self.sell()


class LwmaStrategy(bt.Strategy):
    params = (
        ("short_lwma", 10),  # Kurzfristige LWMA-Periode
        ("long_lwma", 50),   # Langfristige LWMA-Periode
    )

    def __init__(self):
        # LWMA-Indikatoren berechnen
        self.short_lwma = bt.indicators.WeightedMovingAverage(
            self.data.close, period=self.params.short_lwma
        )
        self.long_lwma = bt.indicators.WeightedMovingAverage(
            self.data.close, period=self.params.long_lwma
        )

    def next(self):
        # Kaufen, wenn kurz über lang
        if self.short_lwma > self.long_lwma and not self.position:
            self.buy()
        # Verkaufen, wenn lang über kurz
        elif self.short_lwma < self.long_lwma and self.position:
            self.sell()

# Backtesting durchführen
def run_backtest(data, strategy, seed_capital, commission):
    # Daten für Backtrader vorbereiten
    data_feed = bt.feeds.PandasData(dataname=data)

    # Cerebro (Backtest-Engine) initialisieren
    cerebro = bt.Cerebro()
    cerebro.adddata(data_feed)
    cerebro.addstrategy(strategy)

    # Startkapital festlegen
    cerebro.broker.setcash(seed_capital)
    cerebro.broker.setcommission(commission)  # 0.1% Handelsgebühr

    print("Startkapital: {:.2f}".format(cerebro.broker.getvalue()))

    # Backtest ausführen
    cerebro.run()

    print("Endkapital: {:.2f}".format(cerebro.broker.getvalue()))

    # Ergebnisse plotten
    #cerebro.plot(style='candlestick')

    return cerebro.broker.getvalue()



In [ ]:
    # Parameter
etf_ticker = "SMI"  # Beispiel: S&P 500 ETF
start_date = "2015-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")
seed_capital = 100000
commission = 0.001

etfs = {
    "SMI ETF": "SMI",
    "DAX ETF": "EXS1.DE",
}

etf_data = {}
for name, ticker in etfs.items():
        etfs[name] = fetch_etf_data(ticker, start_date, end_date)
        etfs[name].index = etfs[name].index.tz_localize(None)

# ETF-Daten abrufen
#etf_data = fetch_etf_data(etf_ticker, start_date, end_date)

# Index konvertieren, falls erforderlich
#etf_data.index = etf_data.index.tz_localize(None)  # Zeitstempel entfernen, falls vorhanden

In [ ]:

# Backtest ausführen
#run_backtest(etf_data, SmaStrategy, seed_capital)

In [ ]:
#run_backtest(etf_data, LwmaStrategy)

In [ ]:
print(etfs)

In [ ]:
earnings = []
for name, data in etfs.items():
    sma_earnings = run_backtest(data, SmaStrategy, seed_capital, commission)
    earnings.append({"ETF": name, "Strategy": "SMA", "Earnings": sma_earnings, "Seed Capital": seed_capital, "Profit": sma_earnings - seed_capital, "Profit_Percent": sma_earnings / seed_capital * 100 -100})

    lwma_earnings = run_backtest(data, LwmaStrategy, seed_capital, commission)
    earnings.append({"ETF": name, "Strategy": "LWMA", "Earnings": lwma_earnings, "Seed Capital": seed_capital, "Profit": lwma_earnings - seed_capital, "Profit_Percent": lwma_earnings / seed_capital * 100 - 100})

# Ergebnisse in ein DataFrame umwandeln
earnings_df = pd.DataFrame(earnings)

# DataFrame anzeigen
print(earnings_df)